# Importing Header and SoilPrep

In [1]:
import import_ipynb 
from Header import *

importing Jupyter notebook from Header.ipynb


In [2]:
import import_ipynb
from SoilPrep import * 

importing Jupyter notebook from SoilPrep.ipynb


## Loading Data and MetaData

In [3]:
import import_ipynb
from LoadDataMetaData import *

importing Jupyter notebook from LoadDataMetaData.ipynb


In [9]:
import os
os.system('say "your Meta Tree started building"')
import time

In [172]:
# prepare_spec = ['none', 'fod2', 'continuum', 'log']
# prepare_target = ['none', 'minmax']

def find_X(p, n):
    if p == 'fod2':
        X = fod_sampled[n]
    elif p == 'continuum':
        X = sampled_cr[n]
    elif p == 'log':
        X = sampled_log[n]
    else:
        X = sampled_spec[n]
    return X

def find_spec(p, n, m):
    if n == 0:
        if m == 'randomforest' or m == 'cubist' or m == 'gbrt':
            spec = find_X(p, 100)
        else:
            spec = find_X(p,n)            
    else:
        spec = find_X(p,n)
    return spec    


def find_y(t, tp):
    i = target_names.index(t) 
    if tp == 'minmax':
        y = NT[i]
    else:
        y = T[i]
    return y
        
        

In [173]:
from sklearn.metrics import make_scorer, r2_score

model ={}
param_grid ={}
model['mult'] = LinearRegression()
param_grid['mult'] = { 'fit_intercept': [True, False] }

model['ridge'] = KernelRidge()
param_grid['ridge']={'alpha': [ 0.00001,0.00002, 0.00005, 0.0001, 0.0002, 0.0005, 0.001, 0.002, \
                               0.005,  0.01, 0.05, 0.1, 0.5, 1]}

model['plsr'] = PLSRegression()
param_grid['plsr']={'n_components': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]}


model['cubist'] = Cubist(random_state = 42)  
param_grid['cubist'] = { 'n_committees': [5, 10, 15, 20], 'n_rules': [10, 20, 30, 40, 50] }
                        

model['gbrt'] = GradientBoostingRegressor(random_state = 42)
param_grid['gbrt'] = {
    'n_estimators': [20,30,50],        # Number of boosting stages to be used
    'learning_rate': [0.001, 0.01, 0.1],     # Step size shrinkage used in updating weights
    'max_depth': [2, 3],                # Maximum depth of individual trees
    'min_samples_split': [2, 3],        # Minimum number of samples required to split an internal node
    'min_samples_leaf': [1, 2],         # Minimum number of samples required to be at a leaf node
}

model['svr'] = SVR()
param_grid['svr'] = {
    'C': [0.1, 1, 10],              # Regularization parameter
    'kernel': ['linear', 'rbf'],   # Kernel function
    'gamma': [0.01, 0.1, 1],      # Kernel coefficient (only for 'rbf' kernel)
}



In [199]:
def find_core_model_param_on (m, t, p, tp, n):
    Model = model[m]
    X = find_spec(p,n,m)
    y = find_y(t, tp)
    
    row, col = X.shape

    scorer = make_scorer(mean_squared_error, greater_is_better=False)
    cv = RepeatedKFold(n_splits=row, n_repeats=1, random_state=3)
        
    grid_search = GridSearchCV(Model, param_grid=param_grid[m], cv=cv,\
                                scoring= scorer, refit=True, verbose=1, error_score='raise', n_jobs=-1)
    grid_search.fit(X,y)
    
    return (grid_search.best_params_)

def find_tuned_param_on (X_train, y_train, m):
    Model = model[m]
    X = X_train
    y = y_train
    
    row, col = X.shape

    scorer = make_scorer(mean_squared_error, greater_is_better=False)
    cv = RepeatedKFold(n_splits=row, n_repeats=1, random_state=3)
        
    grid_search = GridSearchCV(Model, param_grid=param_grid[m], cv=cv,\
                                scoring= scorer, refit=True, verbose=1, error_score='raise', n_jobs=-1)
    grid_search.fit(X,y)
    
    return (grid_search.best_params_)
    

In [200]:
def create_core_model_on (m, BP):
    if m == 'mult':
        Model = LinearRegression()
        
    elif m == 'ridge':
        Model =  KernelRidge(alpha = BP['alpha'])
        
    elif m == 'plsr':
        Model = PLSRegression(n_components = BP['n_components'])
        
    elif m == 'cubist':
        Model =  Cubist( n_committees = BP['n_committees'], n_rules = BP['n_rules'], random_state = 42) 
    
    elif m == 'gbrt':
        Model = GradientBoostingRegressor(n_estimators = BP['n_estimators'], learning_rate = BP['learning_rate'], \
                                          max_depth = BP['max_depth'], min_samples_split = BP['min_samples_split'], \
                                          min_samples_leaf = BP['min_samples_leaf'],  random_state = 42)
    elif m == 'svr':
        Model = SVR(C = BP['C'], kernel = BP['kernel'], gamma = BP['gamma'])
        
    return Model    
            

In [ ]:
halt_here

In [176]:
bp = find_core_model_param_on ('mult', 'Sand', 'log', 'none', 10)

Fitting 86 folds for each of 2 candidates, totalling 172 fits


In [177]:
bp

{'fit_intercept': False}

In [178]:
mult_model = create_core_model_on ('mult', bp)

In [179]:
mult_model

LinearRegression()

In [165]:
def build_BPtree_for(method_name):
    tree ={}
    start = time.time()
    m = method_name
    #-- code to build tree----
    for t in target_names:
        print('BPtree for: '+ m +' ------> running on: ' + t)
        tree[t] ={}
        for tp in prepare_target:
            tree[t][tp] ={}
            for p in prepare_spec:
                tree[t][tp][p] ={}
                #---- find appropriate initial core model parameters-----------------
                if m == 'cubist' or m == 'gbrt':
                    BP100 = find_core_model_param_on (m, t, p, tp, 100)
                
                for n in nbands_sampling: 
                    tree[t][tp][p][n] ={}
                    Y = tree[t][tp][p][n]
                    if m == 'cubist' or m == 'gbrt':
                        Y['BP'] = BP100
                    else:    
                        BP = find_core_model_param_on (m, t, p, tp, n)
                        print('n_band:', n, '---> target:', t )
                        Y['BP'] = BP
    end = time.time() 
    print('End time - Start time =', (end-start)) 
    
    return (tree.copy())                

In [163]:
#BPtree = {}

In [183]:
#BPtree['ridge'] = build_BPtree_for('ridge')

In [166]:
# with open ('BPtree.pickle', 'wb') as file:
#     pickle.dump(BPtree, file)

In [184]:
# BPtree['plsr'] = build_BPtree_for('plsr')

In [168]:
# with open ('BPtree.pickle', 'wb') as file:
#     pickle.dump(BPtree, file)

In [185]:
# BPtree['svr'] = build_BPtree_for('svr')

In [180]:
# with open ('BPtree.pickle', 'wb') as file:
#     pickle.dump(BPtree, file)

In [186]:
#BPtree['mult'] = build_BPtree_for('mult')

In [182]:
# with open ('BPtree.pickle', 'wb') as file:
#     pickle.dump(BPtree, file)

##  L1_y_pred

In [187]:
def L1_y_pred (spectra, target, method, bp):
    m = method
    (r,c) = spectra.shape
    
    Y_test = target.copy()
    Y_pred = target.copy()
    
    for i in range (0,r): 
        #print('i: ', i)
        full_spec = spectra.copy()
        X_train = full_spec.drop(full_spec.index[i], axis=0)
        X_test = full_spec.iloc[[i],:].copy()
        full_tar = target.copy()
        y_train = full_tar.drop(full_tar.index[i], axis=0)
        y_test = full_tar.iloc[i].copy() 
        
        #---- Model Creation, fitting, and predictions--------
        Model = create_core_model_on (m, bp)
        Model.fit(X_train, y_train)
        y_pred = Model.predict(X_test)
        Y_pred.iloc[i] = y_pred
        
        return (Y_pred)      

## Building HMtree

In [243]:
def build_HMtree_on (method_name, target):
    tree ={}
    bp_tree ={}
    
    start = time.time()
    m = method_name
    t = target
    
    print('tree for: '+ m +' ------> running on: ' + t)
    
    for tp in prepare_target:
        tree[tp] ={}
        bp_tree[tp] ={}
        for p in prepare_spec:
            tree[tp][p] ={}
            bp_tree[tp][p] ={}
            for n in nbands_sampling:
                tree[tp][p][n] ={}
                Y = tree[tp][p][n]
                
                bp_tree[tp][p][n] ={}
                    
                        
                #------ setting spec to appropriate (sampled) spectra----
                
                spec = find_spec(p, n, m)                
                                        
                #---- target selection and normalization ---
                y = find_y(t, tp)
                
                
                #---- performing train-test split----------------------
                X_train, X_test, y_train, y_test = train_test_split(spec, y, test_size= 0.3, random_state=42)
                            
                #----- hypertuning parameter, model creation, fitting and prediction ----
                
                #L1_y_pred (spectra, target, method, bp)
                #bp = BPtree[m][t][tp][p][n]['BP']
                
                X1_train, X1_test, y1_train, y1_test = train_test_split(X_train, y_train, test_size= 0.5, random_state=42)
                
                if m == 'cubist' or m == 'gbrt':
                    bp = find_tuned_param_on (X1_train, y1_train, m)
                    
                else:
                    bp = find_tuned_param_on (X_train, y_train, m)
                
                bp_tree[tp][p][n]['BP'] = bp
                
                print('Method: '+ m + '---> Target: '+ t+ '----> SpecProc: ' +p+ ' ---> n_band: ', n)
                Model = create_core_model_on (m, bp)
                
                Model.fit(X_train, y_train)
                
                y_pred = Model.predict(X_test)
                yhat_pred = Model.predict(X_train)
                    
                #----- Model accuracy evaluation and plotting datas-----------
                            
                Y['test'] = y_test
                Y['testP'] = y_pred
                Y['train'] = y_train
                Y['trainP'] = yhat_pred
                    
                Y['iqrp_test'] = find_iqrp(y_pred, y_test)
                Y['r2_test'] = find_r2(y_pred, y_test)
                Y['rpd_test'] = find_rpd(y_pred, y_test)                    
                Y['rmse_test'] = find_rmse(y_pred, y_test)
                
                Y['r2_train'] = find_r2(yhat_pred, y_train)
                                                                
                                                                         
    end = time.time()                            
    os.system('say "your meta tree finished for one more method"')                            

    print('End time - Start time =', (end-start)) 
    
    return tree.copy(), bp_tree.copy()

In [244]:
def build_HMtree_for (method_name):
    tree ={}
    bp_tree ={}
    
    start = time.time()
    m = method_name
    for t in target_names:
        #print('tree for: '+ m +' ------> running on: ' + t)
        tree[t], bp_tree[t] = build_HMtree_on (m, t)
        
    end = time.time()                            
    os.system('say "your meta tree finished for one more method"')                            
    print('End time - Start time =', (end-start))     
    return tree.copy(), bp_tree.copy()

In [235]:
# HMtree = {}
# BPtree = {}

In [236]:
#HMtree['mult'], BPtree['mult'] = build_HMtree_for('mult')

tree for: mult ------> running on: Sand
Fitting 60 folds for each of 2 candidates, totalling 120 fits
Method: mult---> Target: Sand----> SpecProc: none ---> n_band:  0
Fitting 60 folds for each of 2 candidates, totalling 120 fits
Method: mult---> Target: Sand----> SpecProc: none ---> n_band:  2
Fitting 60 folds for each of 2 candidates, totalling 120 fits
Method: mult---> Target: Sand----> SpecProc: none ---> n_band:  3
Fitting 60 folds for each of 2 candidates, totalling 120 fits
Method: mult---> Target: Sand----> SpecProc: none ---> n_band:  5
Fitting 60 folds for each of 2 candidates, totalling 120 fits
Method: mult---> Target: Sand----> SpecProc: none ---> n_band:  7
Fitting 60 folds for each of 2 candidates, totalling 120 fits
Method: mult---> Target: Sand----> SpecProc: none ---> n_band:  9
Fitting 60 folds for each of 2 candidates, totalling 120 fits
Method: mult---> Target: Sand----> SpecProc: none ---> n_band:  10
Fitting 60 folds for each of 2 candidates, totalling 120 fits
M

Method: mult---> Target: Sand----> SpecProc: fod2 ---> n_band:  80
Fitting 60 folds for each of 2 candidates, totalling 120 fits
Method: mult---> Target: Sand----> SpecProc: fod2 ---> n_band:  90
Fitting 60 folds for each of 2 candidates, totalling 120 fits
Method: mult---> Target: Sand----> SpecProc: fod2 ---> n_band:  100
Fitting 60 folds for each of 2 candidates, totalling 120 fits
Method: mult---> Target: Sand----> SpecProc: continuum ---> n_band:  0
Fitting 60 folds for each of 2 candidates, totalling 120 fits
Method: mult---> Target: Sand----> SpecProc: continuum ---> n_band:  2
Fitting 60 folds for each of 2 candidates, totalling 120 fits
Method: mult---> Target: Sand----> SpecProc: continuum ---> n_band:  3
Fitting 60 folds for each of 2 candidates, totalling 120 fits
Method: mult---> Target: Sand----> SpecProc: continuum ---> n_band:  5
Fitting 60 folds for each of 2 candidates, totalling 120 fits
Method: mult---> Target: Sand----> SpecProc: continuum ---> n_band:  7
Fitting 6

Method: mult---> Target: Sand----> SpecProc: log ---> n_band:  55
Fitting 60 folds for each of 2 candidates, totalling 120 fits
Method: mult---> Target: Sand----> SpecProc: log ---> n_band:  60
Fitting 60 folds for each of 2 candidates, totalling 120 fits
Method: mult---> Target: Sand----> SpecProc: log ---> n_band:  70
Fitting 60 folds for each of 2 candidates, totalling 120 fits
Method: mult---> Target: Sand----> SpecProc: log ---> n_band:  80
Fitting 60 folds for each of 2 candidates, totalling 120 fits
Method: mult---> Target: Sand----> SpecProc: log ---> n_band:  90
Fitting 60 folds for each of 2 candidates, totalling 120 fits
Method: mult---> Target: Sand----> SpecProc: log ---> n_band:  100
Fitting 60 folds for each of 2 candidates, totalling 120 fits
Method: mult---> Target: Sand----> SpecProc: none ---> n_band:  0
Fitting 60 folds for each of 2 candidates, totalling 120 fits
Method: mult---> Target: Sand----> SpecProc: none ---> n_band:  2
Fitting 60 folds for each of 2 candid

Method: mult---> Target: Sand----> SpecProc: fod2 ---> n_band:  45
Fitting 60 folds for each of 2 candidates, totalling 120 fits
Method: mult---> Target: Sand----> SpecProc: fod2 ---> n_band:  50
Fitting 60 folds for each of 2 candidates, totalling 120 fits
Method: mult---> Target: Sand----> SpecProc: fod2 ---> n_band:  55
Fitting 60 folds for each of 2 candidates, totalling 120 fits
Method: mult---> Target: Sand----> SpecProc: fod2 ---> n_band:  60
Fitting 60 folds for each of 2 candidates, totalling 120 fits
Method: mult---> Target: Sand----> SpecProc: fod2 ---> n_band:  70
Fitting 60 folds for each of 2 candidates, totalling 120 fits
Method: mult---> Target: Sand----> SpecProc: fod2 ---> n_band:  80
Fitting 60 folds for each of 2 candidates, totalling 120 fits
Method: mult---> Target: Sand----> SpecProc: fod2 ---> n_band:  90
Fitting 60 folds for each of 2 candidates, totalling 120 fits
Method: mult---> Target: Sand----> SpecProc: fod2 ---> n_band:  100
Fitting 60 folds for each of 

Method: mult---> Target: Sand----> SpecProc: log ---> n_band:  37
Fitting 60 folds for each of 2 candidates, totalling 120 fits
Method: mult---> Target: Sand----> SpecProc: log ---> n_band:  39
Fitting 60 folds for each of 2 candidates, totalling 120 fits
Method: mult---> Target: Sand----> SpecProc: log ---> n_band:  40
Fitting 60 folds for each of 2 candidates, totalling 120 fits
Method: mult---> Target: Sand----> SpecProc: log ---> n_band:  45
Fitting 60 folds for each of 2 candidates, totalling 120 fits
Method: mult---> Target: Sand----> SpecProc: log ---> n_band:  50
Fitting 60 folds for each of 2 candidates, totalling 120 fits
Method: mult---> Target: Sand----> SpecProc: log ---> n_band:  55
Fitting 60 folds for each of 2 candidates, totalling 120 fits
Method: mult---> Target: Sand----> SpecProc: log ---> n_band:  60
Fitting 60 folds for each of 2 candidates, totalling 120 fits
Method: mult---> Target: Sand----> SpecProc: log ---> n_band:  70
Fitting 60 folds for each of 2 candida

...

Method: mult---> Target: TOC----> SpecProc: continuum ---> n_band:  80
Fitting 60 folds for each of 2 candidates, totalling 120 fits
Method: mult---> Target: TOC----> SpecProc: continuum ---> n_band:  90
Fitting 60 folds for each of 2 candidates, totalling 120 fits
Method: mult---> Target: TOC----> SpecProc: continuum ---> n_band:  100
Fitting 60 folds for each of 2 candidates, totalling 120 fits
Method: mult---> Target: TOC----> SpecProc: log ---> n_band:  0
Fitting 60 folds for each of 2 candidates, totalling 120 fits
Method: mult---> Target: TOC----> SpecProc: log ---> n_band:  2
Fitting 60 folds for each of 2 candidates, totalling 120 fits
Method: mult---> Target: TOC----> SpecProc: log ---> n_band:  3
Fitting 60 folds for each of 2 candidates, totalling 120 fits
Method: mult---> Target: TOC----> SpecProc: log ---> n_band:  5
Fitting 60 folds for each of 2 candidates, totalling 120 fits
Method: mult---> Target: TOC----> SpecProc: log ---> n_band:  7
Fitting 60 folds for each of 2 c

Method: mult---> Target: CaCO3----> SpecProc: none ---> n_band:  60
Fitting 60 folds for each of 2 candidates, totalling 120 fits
Method: mult---> Target: CaCO3----> SpecProc: none ---> n_band:  70
Fitting 60 folds for each of 2 candidates, totalling 120 fits
Method: mult---> Target: CaCO3----> SpecProc: none ---> n_band:  80
Fitting 60 folds for each of 2 candidates, totalling 120 fits
Method: mult---> Target: CaCO3----> SpecProc: none ---> n_band:  90
Fitting 60 folds for each of 2 candidates, totalling 120 fits
Method: mult---> Target: CaCO3----> SpecProc: none ---> n_band:  100
Fitting 60 folds for each of 2 candidates, totalling 120 fits
Method: mult---> Target: CaCO3----> SpecProc: fod2 ---> n_band:  0
Fitting 60 folds for each of 2 candidates, totalling 120 fits
Method: mult---> Target: CaCO3----> SpecProc: fod2 ---> n_band:  2
Fitting 60 folds for each of 2 candidates, totalling 120 fits
Method: mult---> Target: CaCO3----> SpecProc: fod2 ---> n_band:  3
Fitting 60 folds for eac

Method: mult---> Target: CaCO3----> SpecProc: continuum ---> n_band:  45
Fitting 60 folds for each of 2 candidates, totalling 120 fits
Method: mult---> Target: CaCO3----> SpecProc: continuum ---> n_band:  50
Fitting 60 folds for each of 2 candidates, totalling 120 fits
Method: mult---> Target: CaCO3----> SpecProc: continuum ---> n_band:  55
Fitting 60 folds for each of 2 candidates, totalling 120 fits
Method: mult---> Target: CaCO3----> SpecProc: continuum ---> n_band:  60
Fitting 60 folds for each of 2 candidates, totalling 120 fits
Method: mult---> Target: CaCO3----> SpecProc: continuum ---> n_band:  70
Fitting 60 folds for each of 2 candidates, totalling 120 fits
Method: mult---> Target: CaCO3----> SpecProc: continuum ---> n_band:  80
Fitting 60 folds for each of 2 candidates, totalling 120 fits
Method: mult---> Target: CaCO3----> SpecProc: continuum ---> n_band:  90
Fitting 60 folds for each of 2 candidates, totalling 120 fits
Method: mult---> Target: CaCO3----> SpecProc: continuum

Method: mult---> Target: CaCO3----> SpecProc: none ---> n_band:  39
Fitting 60 folds for each of 2 candidates, totalling 120 fits
Method: mult---> Target: CaCO3----> SpecProc: none ---> n_band:  40
Fitting 60 folds for each of 2 candidates, totalling 120 fits
Method: mult---> Target: CaCO3----> SpecProc: none ---> n_band:  45
Fitting 60 folds for each of 2 candidates, totalling 120 fits
Method: mult---> Target: CaCO3----> SpecProc: none ---> n_band:  50
Fitting 60 folds for each of 2 candidates, totalling 120 fits
Method: mult---> Target: CaCO3----> SpecProc: none ---> n_band:  55
Fitting 60 folds for each of 2 candidates, totalling 120 fits
Method: mult---> Target: CaCO3----> SpecProc: none ---> n_band:  60
Fitting 60 folds for each of 2 candidates, totalling 120 fits
Method: mult---> Target: CaCO3----> SpecProc: none ---> n_band:  70
Fitting 60 folds for each of 2 candidates, totalling 120 fits
Method: mult---> Target: CaCO3----> SpecProc: none ---> n_band:  80
Fitting 60 folds for e

Method: mult---> Target: CaCO3----> SpecProc: continuum ---> n_band:  33
Fitting 60 folds for each of 2 candidates, totalling 120 fits
Method: mult---> Target: CaCO3----> SpecProc: continuum ---> n_band:  35
Fitting 60 folds for each of 2 candidates, totalling 120 fits
Method: mult---> Target: CaCO3----> SpecProc: continuum ---> n_band:  37
Fitting 60 folds for each of 2 candidates, totalling 120 fits
Method: mult---> Target: CaCO3----> SpecProc: continuum ---> n_band:  39
Fitting 60 folds for each of 2 candidates, totalling 120 fits
Method: mult---> Target: CaCO3----> SpecProc: continuum ---> n_band:  40
Fitting 60 folds for each of 2 candidates, totalling 120 fits
Method: mult---> Target: CaCO3----> SpecProc: continuum ---> n_band:  45
Fitting 60 folds for each of 2 candidates, totalling 120 fits
Method: mult---> Target: CaCO3----> SpecProc: continuum ---> n_band:  50
Fitting 60 folds for each of 2 candidates, totalling 120 fits
Method: mult---> Target: CaCO3----> SpecProc: continuum

In [237]:
# with open ('HMtree.pickle', 'wb') as file:
#     pickle.dump(HMtree, file)
    
# with open ('BPtree.pickle', 'wb') as file:
#     pickle.dump(BPtree, file)    

In [238]:
best_score_on (HMtree, 'Clay', 'mult', 'r2')

['r2', 0.75, 'Spec:', 'log', 'bands:', 11, 'Tar:', 'minmax']

In [239]:
# HMtree['plsr'], BPtree['plsr'] = build_HMtree_for('plsr')
# HMtree['svr'] , BPtree['svr']= build_HMtree_for('svr')
# HMtree['ridge'], BPtree['ridge'] = build_HMtree_for('ridge')

tree for: plsr ------> running on: Sand
Fitting 60 folds for each of 10 candidates, totalling 600 fits
Method: plsr---> Target: Sand----> SpecProc: none ---> n_band:  0
Fitting 60 folds for each of 10 candidates, totalling 600 fits
Method: plsr---> Target: Sand----> SpecProc: none ---> n_band:  2
Fitting 60 folds for each of 10 candidates, totalling 600 fits
Method: plsr---> Target: Sand----> SpecProc: none ---> n_band:  3
Fitting 60 folds for each of 10 candidates, totalling 600 fits
Method: plsr---> Target: Sand----> SpecProc: none ---> n_band:  5
Fitting 60 folds for each of 10 candidates, totalling 600 fits
Method: plsr---> Target: Sand----> SpecProc: none ---> n_band:  7
Fitting 60 folds for each of 10 candidates, totalling 600 fits
Method: plsr---> Target: Sand----> SpecProc: none ---> n_band:  9
Fitting 60 folds for each of 10 candidates, totalling 600 fits
Method: plsr---> Target: Sand----> SpecProc: none ---> n_band:  10
Fitting 60 folds for each of 10 candidates, totalling 60

Method: plsr---> Target: Sand----> SpecProc: fod2 ---> n_band:  80
Fitting 60 folds for each of 10 candidates, totalling 600 fits
Method: plsr---> Target: Sand----> SpecProc: fod2 ---> n_band:  90
Fitting 60 folds for each of 10 candidates, totalling 600 fits
Method: plsr---> Target: Sand----> SpecProc: fod2 ---> n_band:  100
Fitting 60 folds for each of 10 candidates, totalling 600 fits
Method: plsr---> Target: Sand----> SpecProc: continuum ---> n_band:  0
Fitting 60 folds for each of 10 candidates, totalling 600 fits
Method: plsr---> Target: Sand----> SpecProc: continuum ---> n_band:  2
Fitting 60 folds for each of 10 candidates, totalling 600 fits
Method: plsr---> Target: Sand----> SpecProc: continuum ---> n_band:  3
Fitting 60 folds for each of 10 candidates, totalling 600 fits
Method: plsr---> Target: Sand----> SpecProc: continuum ---> n_band:  5
Fitting 60 folds for each of 10 candidates, totalling 600 fits
Method: plsr---> Target: Sand----> SpecProc: continuum ---> n_band:  7
Fi

Method: plsr---> Target: Sand----> SpecProc: log ---> n_band:  55
Fitting 60 folds for each of 10 candidates, totalling 600 fits
Method: plsr---> Target: Sand----> SpecProc: log ---> n_band:  60
Fitting 60 folds for each of 10 candidates, totalling 600 fits
Method: plsr---> Target: Sand----> SpecProc: log ---> n_band:  70
Fitting 60 folds for each of 10 candidates, totalling 600 fits
Method: plsr---> Target: Sand----> SpecProc: log ---> n_band:  80
Fitting 60 folds for each of 10 candidates, totalling 600 fits
Method: plsr---> Target: Sand----> SpecProc: log ---> n_band:  90
Fitting 60 folds for each of 10 candidates, totalling 600 fits
Method: plsr---> Target: Sand----> SpecProc: log ---> n_band:  100
Fitting 60 folds for each of 10 candidates, totalling 600 fits
Method: plsr---> Target: Sand----> SpecProc: none ---> n_band:  0
Fitting 60 folds for each of 10 candidates, totalling 600 fits
Method: plsr---> Target: Sand----> SpecProc: none ---> n_band:  2
Fitting 60 folds for each of 1

Method: plsr---> Target: Sand----> SpecProc: fod2 ---> n_band:  45
Fitting 60 folds for each of 10 candidates, totalling 600 fits
Method: plsr---> Target: Sand----> SpecProc: fod2 ---> n_band:  50
Fitting 60 folds for each of 10 candidates, totalling 600 fits
Method: plsr---> Target: Sand----> SpecProc: fod2 ---> n_band:  55
Fitting 60 folds for each of 10 candidates, totalling 600 fits
Method: plsr---> Target: Sand----> SpecProc: fod2 ---> n_band:  60
Fitting 60 folds for each of 10 candidates, totalling 600 fits
Method: plsr---> Target: Sand----> SpecProc: fod2 ---> n_band:  70
Fitting 60 folds for each of 10 candidates, totalling 600 fits
Method: plsr---> Target: Sand----> SpecProc: fod2 ---> n_band:  80
Fitting 60 folds for each of 10 candidates, totalling 600 fits
Method: plsr---> Target: Sand----> SpecProc: fod2 ---> n_band:  90
Fitting 60 folds for each of 10 candidates, totalling 600 fits
Method: plsr---> Target: Sand----> SpecProc: fod2 ---> n_band:  100
Fitting 60 folds for e

Method: plsr---> Target: Sand----> SpecProc: log ---> n_band:  35
Fitting 60 folds for each of 10 candidates, totalling 600 fits
Method: plsr---> Target: Sand----> SpecProc: log ---> n_band:  37
Fitting 60 folds for each of 10 candidates, totalling 600 fits
Method: plsr---> Target: Sand----> SpecProc: log ---> n_band:  39
Fitting 60 folds for each of 10 candidates, totalling 600 fits
Method: plsr---> Target: Sand----> SpecProc: log ---> n_band:  40
Fitting 60 folds for each of 10 candidates, totalling 600 fits
Method: plsr---> Target: Sand----> SpecProc: log ---> n_band:  45
Fitting 60 folds for each of 10 candidates, totalling 600 fits
Method: plsr---> Target: Sand----> SpecProc: log ---> n_band:  50
Fitting 60 folds for each of 10 candidates, totalling 600 fits
Method: plsr---> Target: Sand----> SpecProc: log ---> n_band:  55
Fitting 60 folds for each of 10 candidates, totalling 600 fits
Method: plsr---> Target: Sand----> SpecProc: log ---> n_band:  60
Fitting 60 folds for each of 10

...

Method: ridge---> Target: CaCO3----> SpecProc: none ---> n_band:  19
Fitting 60 folds for each of 14 candidates, totalling 840 fits
Method: ridge---> Target: CaCO3----> SpecProc: none ---> n_band:  20
Fitting 60 folds for each of 14 candidates, totalling 840 fits
Method: ridge---> Target: CaCO3----> SpecProc: none ---> n_band:  21
Fitting 60 folds for each of 14 candidates, totalling 840 fits
Method: ridge---> Target: CaCO3----> SpecProc: none ---> n_band:  23
Fitting 60 folds for each of 14 candidates, totalling 840 fits
Method: ridge---> Target: CaCO3----> SpecProc: none ---> n_band:  25
Fitting 60 folds for each of 14 candidates, totalling 840 fits
Method: ridge---> Target: CaCO3----> SpecProc: none ---> n_band:  27
Fitting 60 folds for each of 14 candidates, totalling 840 fits
Method: ridge---> Target: CaCO3----> SpecProc: none ---> n_band:  29
Fitting 60 folds for each of 14 candidates, totalling 840 fits
Method: ridge---> Target: CaCO3----> SpecProc: none ---> n_band:  30
Fitting

Method: ridge---> Target: CaCO3----> SpecProc: continuum ---> n_band:  11
Fitting 60 folds for each of 14 candidates, totalling 840 fits
Method: ridge---> Target: CaCO3----> SpecProc: continuum ---> n_band:  13
Fitting 60 folds for each of 14 candidates, totalling 840 fits
Method: ridge---> Target: CaCO3----> SpecProc: continuum ---> n_band:  15
Fitting 60 folds for each of 14 candidates, totalling 840 fits
Method: ridge---> Target: CaCO3----> SpecProc: continuum ---> n_band:  17
Fitting 60 folds for each of 14 candidates, totalling 840 fits
Method: ridge---> Target: CaCO3----> SpecProc: continuum ---> n_band:  19
Fitting 60 folds for each of 14 candidates, totalling 840 fits
Method: ridge---> Target: CaCO3----> SpecProc: continuum ---> n_band:  20
Fitting 60 folds for each of 14 candidates, totalling 840 fits
Method: ridge---> Target: CaCO3----> SpecProc: continuum ---> n_band:  21
Fitting 60 folds for each of 14 candidates, totalling 840 fits
Method: ridge---> Target: CaCO3----> Spec

Method: ridge---> Target: CaCO3----> SpecProc: none ---> n_band:  5
Fitting 60 folds for each of 14 candidates, totalling 840 fits
Method: ridge---> Target: CaCO3----> SpecProc: none ---> n_band:  7
Fitting 60 folds for each of 14 candidates, totalling 840 fits
Method: ridge---> Target: CaCO3----> SpecProc: none ---> n_band:  9
Fitting 60 folds for each of 14 candidates, totalling 840 fits
Method: ridge---> Target: CaCO3----> SpecProc: none ---> n_band:  10
Fitting 60 folds for each of 14 candidates, totalling 840 fits
Method: ridge---> Target: CaCO3----> SpecProc: none ---> n_band:  11
Fitting 60 folds for each of 14 candidates, totalling 840 fits
Method: ridge---> Target: CaCO3----> SpecProc: none ---> n_band:  13
Fitting 60 folds for each of 14 candidates, totalling 840 fits
Method: ridge---> Target: CaCO3----> SpecProc: none ---> n_band:  15
Fitting 60 folds for each of 14 candidates, totalling 840 fits
Method: ridge---> Target: CaCO3----> SpecProc: none ---> n_band:  17
Fitting 60

Method: ridge---> Target: CaCO3----> SpecProc: continuum ---> n_band:  0
Fitting 60 folds for each of 14 candidates, totalling 840 fits
Method: ridge---> Target: CaCO3----> SpecProc: continuum ---> n_band:  2
Fitting 60 folds for each of 14 candidates, totalling 840 fits
Method: ridge---> Target: CaCO3----> SpecProc: continuum ---> n_band:  3
Fitting 60 folds for each of 14 candidates, totalling 840 fits
Method: ridge---> Target: CaCO3----> SpecProc: continuum ---> n_band:  5
Fitting 60 folds for each of 14 candidates, totalling 840 fits
Method: ridge---> Target: CaCO3----> SpecProc: continuum ---> n_band:  7
Fitting 60 folds for each of 14 candidates, totalling 840 fits
Method: ridge---> Target: CaCO3----> SpecProc: continuum ---> n_band:  9
Fitting 60 folds for each of 14 candidates, totalling 840 fits
Method: ridge---> Target: CaCO3----> SpecProc: continuum ---> n_band:  10
Fitting 60 folds for each of 14 candidates, totalling 840 fits
Method: ridge---> Target: CaCO3----> SpecProc: 

Method: ridge---> Target: CaCO3----> SpecProc: log ---> n_band:  70
Fitting 60 folds for each of 14 candidates, totalling 840 fits
Method: ridge---> Target: CaCO3----> SpecProc: log ---> n_band:  80
Fitting 60 folds for each of 14 candidates, totalling 840 fits
Method: ridge---> Target: CaCO3----> SpecProc: log ---> n_band:  90
Fitting 60 folds for each of 14 candidates, totalling 840 fits
Method: ridge---> Target: CaCO3----> SpecProc: log ---> n_band:  100
End time - Start time = 262.84553933143616
End time - Start time = 1320.6873433589935


In [241]:
# with open ('HMtree.pickle', 'wb') as file:
#     pickle.dump(HMtree, file)
    
# with open ('BPtree.pickle', 'wb') as file:
#     pickle.dump(BPtree, file)    

In [ ]:
HMtree['cubist'], BPtree['cubist'] = build_HMtree_for('cubist')

tree for: cubist ------> running on: Sand
Fitting 30 folds for each of 20 candidates, totalling 600 fits
Method: cubist---> Target: Sand----> SpecProc: none ---> n_band:  0
Fitting 30 folds for each of 20 candidates, totalling 600 fits
Method: cubist---> Target: Sand----> SpecProc: none ---> n_band:  2
Fitting 30 folds for each of 20 candidates, totalling 600 fits
Method: cubist---> Target: Sand----> SpecProc: none ---> n_band:  3
Fitting 30 folds for each of 20 candidates, totalling 600 fits
Method: cubist---> Target: Sand----> SpecProc: none ---> n_band:  5
Fitting 30 folds for each of 20 candidates, totalling 600 fits
Method: cubist---> Target: Sand----> SpecProc: none ---> n_band:  7
Fitting 30 folds for each of 20 candidates, totalling 600 fits
Method: cubist---> Target: Sand----> SpecProc: none ---> n_band:  9
Fitting 30 folds for each of 20 candidates, totalling 600 fits
Method: cubist---> Target: Sand----> SpecProc: none ---> n_band:  10
Fitting 30 folds for each of 20 candidat

C:\Users\abhis\anaconda3\lib\site-packages\joblib\externals\loky\process_executor.py:700: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


Method: cubist---> Target: Sand----> SpecProc: none ---> n_band:  90
Fitting 30 folds for each of 20 candidates, totalling 600 fits
Method: cubist---> Target: Sand----> SpecProc: none ---> n_band:  100
Fitting 30 folds for each of 20 candidates, totalling 600 fits
Method: cubist---> Target: Sand----> SpecProc: fod2 ---> n_band:  0
Fitting 30 folds for each of 20 candidates, totalling 600 fits
Method: cubist---> Target: Sand----> SpecProc: fod2 ---> n_band:  2
Fitting 30 folds for each of 20 candidates, totalling 600 fits
Method: cubist---> Target: Sand----> SpecProc: fod2 ---> n_band:  3
Fitting 30 folds for each of 20 candidates, totalling 600 fits
Method: cubist---> Target: Sand----> SpecProc: fod2 ---> n_band:  5
Fitting 30 folds for each of 20 candidates, totalling 600 fits
Method: cubist---> Target: Sand----> SpecProc: fod2 ---> n_band:  7
Fitting 30 folds for each of 20 candidates, totalling 600 fits
Method: cubist---> Target: Sand----> SpecProc: fod2 ---> n_band:  9
Fitting 30 f

C:\Users\abhis\anaconda3\lib\site-packages\joblib\externals\loky\process_executor.py:700: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


Method: cubist---> Target: Sand----> SpecProc: fod2 ---> n_band:  60
Fitting 30 folds for each of 20 candidates, totalling 600 fits
Method: cubist---> Target: Sand----> SpecProc: fod2 ---> n_band:  70
Fitting 30 folds for each of 20 candidates, totalling 600 fits
Method: cubist---> Target: Sand----> SpecProc: fod2 ---> n_band:  80
Fitting 30 folds for each of 20 candidates, totalling 600 fits
Method: cubist---> Target: Sand----> SpecProc: fod2 ---> n_band:  90
Fitting 30 folds for each of 20 candidates, totalling 600 fits
Method: cubist---> Target: Sand----> SpecProc: fod2 ---> n_band:  100
Fitting 30 folds for each of 20 candidates, totalling 600 fits
Method: cubist---> Target: Sand----> SpecProc: continuum ---> n_band:  0
Fitting 30 folds for each of 20 candidates, totalling 600 fits
Method: cubist---> Target: Sand----> SpecProc: continuum ---> n_band:  2
Fitting 30 folds for each of 20 candidates, totalling 600 fits
Method: cubist---> Target: Sand----> SpecProc: continuum ---> n_ban

...

Method: cubist---> Target: Clay----> SpecProc: none ---> n_band:  29
Fitting 30 folds for each of 20 candidates, totalling 600 fits
Method: cubist---> Target: Clay----> SpecProc: none ---> n_band:  30
Fitting 30 folds for each of 20 candidates, totalling 600 fits
Method: cubist---> Target: Clay----> SpecProc: none ---> n_band:  31
Fitting 30 folds for each of 20 candidates, totalling 600 fits
Method: cubist---> Target: Clay----> SpecProc: none ---> n_band:  33
Fitting 30 folds for each of 20 candidates, totalling 600 fits
Method: cubist---> Target: Clay----> SpecProc: none ---> n_band:  35
Fitting 30 folds for each of 20 candidates, totalling 600 fits
Method: cubist---> Target: Clay----> SpecProc: none ---> n_band:  37
Fitting 30 folds for each of 20 candidates, totalling 600 fits
Method: cubist---> Target: Clay----> SpecProc: none ---> n_band:  39
Fitting 30 folds for each of 20 candidates, totalling 600 fits
Method: cubist---> Target: Clay----> SpecProc: none ---> n_band:  40
Fitting

C:\Users\abhis\anaconda3\lib\site-packages\joblib\externals\loky\process_executor.py:700: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


Method: cubist---> Target: Clay----> SpecProc: fod2 ---> n_band:  10
Fitting 30 folds for each of 20 candidates, totalling 600 fits
Method: cubist---> Target: Clay----> SpecProc: fod2 ---> n_band:  11
Fitting 30 folds for each of 20 candidates, totalling 600 fits
Method: cubist---> Target: Clay----> SpecProc: fod2 ---> n_band:  13
Fitting 30 folds for each of 20 candidates, totalling 600 fits


C:\Users\abhis\anaconda3\lib\site-packages\joblib\externals\loky\process_executor.py:700: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


Method: cubist---> Target: Clay----> SpecProc: fod2 ---> n_band:  15
Fitting 30 folds for each of 20 candidates, totalling 600 fits
Method: cubist---> Target: Clay----> SpecProc: fod2 ---> n_band:  17
Fitting 30 folds for each of 20 candidates, totalling 600 fits
Method: cubist---> Target: Clay----> SpecProc: fod2 ---> n_band:  19
Fitting 30 folds for each of 20 candidates, totalling 600 fits
Method: cubist---> Target: Clay----> SpecProc: fod2 ---> n_band:  20
Fitting 30 folds for each of 20 candidates, totalling 600 fits
Method: cubist---> Target: Clay----> SpecProc: fod2 ---> n_band:  21
Fitting 30 folds for each of 20 candidates, totalling 600 fits
Method: cubist---> Target: Clay----> SpecProc: fod2 ---> n_band:  23
Fitting 30 folds for each of 20 candidates, totalling 600 fits
Method: cubist---> Target: Clay----> SpecProc: fod2 ---> n_band:  25
Fitting 30 folds for each of 20 candidates, totalling 600 fits
Method: cubist---> Target: Clay----> SpecProc: fod2 ---> n_band:  27
Fitting

In [ ]:
with open ('HMtree.pickle', 'wb') as file:
    pickle.dump(HMtree, file)
    
with open ('BPtree.pickle', 'wb') as file:
    pickle.dump(BPtree, file)    

In [219]:
def best_score_on (HMtree, target, method, scorer):
    t=target
    m= method
    
    best_score = -1
    #best_n_comp = 'NA'
    
    
    for tp in prepare_target:
        for p in prepare_spec:
            for n in nbands_sampling:
                Y = HMtree[m][t][tp][p][n]
                #r2_train = Y['r2_train']
                r2_test = Y['r2_test']
                    
                if scorer == 'iqrp':
                    cur_score = Y['iqrp_test']
                elif scorer == 'rpd':
                    cur_score = Y['rpd_test']
                else:
                    cur_score = Y['r2_test']  
                    
                if cur_score > best_score:
                    best_score = cur_score
                    best_tp = tp
                    best_n = n
                    best_p = p
                            
    param_list = [scorer, np.round(best_score,2), 'Spec:', best_p, 'bands:', best_n, 'Tar:', best_tp]                                 
    return (param_list)          

In [220]:
def best_score_for (HMtree, target, scorer):
    
    for method in ml_methods:
        param_list= best_score_on (HMtree, target, method, scorer)
        print('For:'+target+'->', param_list, ':'+method)
     
    return

In [221]:
ml_methods = ['mult', 'ridge', 'plsr', 'svr']

In [229]:
#Plotting Model Accuracy (ipywidgets)
def plot_model_acc (method, target, target_preprocessing, spec_preprocessing, n_bands):
    
    m = method
    t = target
    
    i = target_names.index(target)
    
    
    p = spec_preprocessing
    n = n_bands
    tp = target_preprocessing
    
    Y = HMtree[m][t][tp][p][n]
    
    y_test = Y['test']
    y_pred = Y['testP']
#     y_train = Y['train']
#     yhat_pred = Y['trainP']
    
    if m == 'plsr':
        y_pred = y_pred[:,0]
#       n_com = Y['n_comp']
        
        #yhat_pred = yhat_pred[:,0]
    
    
    iqrp_test = Y['iqrp_test']
    r2_test = Y['r2_test']
    rpd_test = Y['rpd_test']
    
#     iqrp_train = find_iqrp(yhat_pred, y_train)
#     r2_train = find_r2(yhat_pred, y_train)
#     rpd_train = find_rpd(yhat_pred, y_train)
    
    y_tp = pd.DataFrame({'actual':y_test.values, 'predic': y_pred})
    z = np.polyfit(y_test, y_pred, 1)
    
#     yhat_tp = pd.DataFrame({'actual':y_train.values, 'predic': yhat_pred})
#     zhat = np.polyfit(y_train, yhat_pred, 1)
    
    fig, axes = plt.subplots(1,2, figsize=(18,8))
    
    #with plt.style.context(('ggplot')): ---- PLOT of test-prediction --------------------------------------
    y_tp.plot.scatter(ax= axes[0], x="actual", y="predic", alpha=0.8, color = clr[i], edgecolors='k')
    axes[0].plot(y_test, np.polyval(z, y_test),  c='blue', linewidth=1)
    axes[0].plot(y_test, y_test, color='green', linewidth=1)
    axes[0].tick_params(axis='both', labelsize=10)
    axes[0].text(0.05, 0.95, target_names[i]+' (Test Data)', transform=axes[0].transAxes, fontsize = 20, color = clr[i])
    axes[0].text(0.05, 0.90, 'IQRP ={:.2f}'.format(iqrp_test), transform=axes[0].transAxes, fontsize = 16)
    axes[0].text(0.05, 0.85, 'RPD ={:.2f}'.format(rpd_test), transform=axes[0].transAxes, fontsize = 16)
    axes[0].text(0.05, 0.80, 'R2 ={:.2f}'.format(np.round(r2_test,3)), transform=axes[0].transAxes, fontsize = 16)
    axes[0].text(0.95, 0.15, 'Method: '+method, transform=axes[0].transAxes, 
                    horizontalalignment='right', fontsize = 20)
    
#     if method == 'plsr':
#         axes[0].text(0.95, 0.05, 'n_component={:.2f}'.format(n_com), transform=axes[0].transAxes, 
#                     horizontalalignment='right', fontsize = 12)
    
     #------------------------ Ploting R2 Vs N_bands-----------------------------------
    
    #--- data for creating n_bands vs r2 scores plot -------------------------------
    
    pY = HMtree[m][t]['none'][p]
    pYN = HMtree[m][t]['minmax'][p]
    
    
    X = nbands_sampling
    
    Yr2 = []
    YNr2 = []
    
    
    for j in X:
        Yr2.append(pY[j]['r2_test']) 
        YNr2.append(pYN[j]['r2_test'])
        
        
   
    j = 0
    while j < len(Yr2): 
        if Yr2[j] <= 0:
            Yr2[j] = 0
        if YNr2[j] <= 0:
            YNr2[j] = 0
        
        j = j + 1
 
    #------------------------------ STEM PLOT ---- for accuracy Vs n_bands ---------------------------------- 
    if tp == 'none':
        axes[1].stem(X,Yr2)
        axes[1].tick_params(axis='both', labelsize=10)
        axes[1].text(0.00, 1.01,  target_names[i], transform=axes[1].transAxes,fontsize = 20, color = clr[i])
        axes[1].text(0.99, 1.01, 'Spec_prep: '+ p, transform=axes[1].transAxes, horizontalalignment='right', fontsize = 16)
        axes[1].text(0.17, 1.01, '(none)', transform=axes[1].transAxes, horizontalalignment='left', fontsize = 16)
        axes[1].text(0.60, -0.1, 'n_bands', transform=axes[1].transAxes, horizontalalignment='right', fontsize = 16)
        axes[1].text(-0.1, 0.5, 'R2 Scores', horizontalalignment='left', verticalalignment='center', \
                    rotation='vertical', transform=axes[1].transAxes, fontsize = 16)
        
    else:
        axes[1].stem(X,YNr2)
        axes[1].tick_params(axis='both', labelsize=10)
        axes[1].text(0.00, 1.01,  target_names[i], transform=axes[1].transAxes,fontsize = 20, color = clr[i])
        axes[1].text(0.99, 1.01, 'Spec_prep: '+ p, transform=axes[1].transAxes, horizontalalignment='right', fontsize = 16)
        axes[1].text(0.17, 1.01, '(minmax)', transform=axes[1].transAxes, horizontalalignment='left', fontsize = 16)
        axes[1].text(0.60, -0.1, 'n_bands', transform=axes[1].transAxes, horizontalalignment='right', fontsize = 16)
        axes[1].text(-0.1, 0.5, 'R2 Scores', horizontalalignment='left', verticalalignment='center', \
                    rotation='vertical', transform=axes[1].transAxes, fontsize = 16)
        
   
    return    

In [222]:
best_score_for (HMtree, 'Sand', 'r2')

For:Sand-> ['r2', 0.64, 'Spec:', 'log', 'bands:', 7, 'Tar:', 'none'] :mult
For:Sand-> ['r2', 0.6, 'Spec:', 'log', 'bands:', 7, 'Tar:', 'none'] :ridge
For:Sand-> ['r2', 0.64, 'Spec:', 'log', 'bands:', 7, 'Tar:', 'minmax'] :plsr
For:Sand-> ['r2', 0.63, 'Spec:', 'log', 'bands:', 55, 'Tar:', 'minmax'] :svr


In [223]:
best_score_for (HMtree, 'Silt', 'r2')

For:Silt-> ['r2', 0.36, 'Spec:', 'none', 'bands:', 5, 'Tar:', 'none'] :mult
For:Silt-> ['r2', 0.36, 'Spec:', 'none', 'bands:', 5, 'Tar:', 'none'] :ridge
For:Silt-> ['r2', 0.36, 'Spec:', 'none', 'bands:', 5, 'Tar:', 'minmax'] :plsr
For:Silt-> ['r2', 0.58, 'Spec:', 'log', 'bands:', 45, 'Tar:', 'minmax'] :svr


In [224]:
best_score_for (HMtree, 'Clay', 'r2')

For:Clay-> ['r2', 0.75, 'Spec:', 'log', 'bands:', 11, 'Tar:', 'minmax'] :mult
For:Clay-> ['r2', 0.72, 'Spec:', 'log', 'bands:', 11, 'Tar:', 'minmax'] :ridge
For:Clay-> ['r2', 0.71, 'Spec:', 'log', 'bands:', 7, 'Tar:', 'none'] :plsr
For:Clay-> ['r2', 0.69, 'Spec:', 'log', 'bands:', 29, 'Tar:', 'minmax'] :svr


In [225]:
best_score_for (HMtree, 'TOC', 'r2')

For:TOC-> ['r2', 0.61, 'Spec:', 'log', 'bands:', 19, 'Tar:', 'minmax'] :mult
For:TOC-> ['r2', 0.58, 'Spec:', 'log', 'bands:', 31, 'Tar:', 'none'] :ridge
For:TOC-> ['r2', 0.57, 'Spec:', 'log', 'bands:', 35, 'Tar:', 'none'] :plsr
For:TOC-> ['r2', 0.58, 'Spec:', 'log', 'bands:', 0, 'Tar:', 'none'] :svr


In [226]:
best_score_for (HMtree, 'CaCO3', 'r2')

For:CaCO3-> ['r2', 0.67, 'Spec:', 'log', 'bands:', 7, 'Tar:', 'minmax'] :mult
For:CaCO3-> ['r2', 0.63, 'Spec:', 'none', 'bands:', 7, 'Tar:', 'minmax'] :ridge
For:CaCO3-> ['r2', 0.67, 'Spec:', 'log', 'bands:', 7, 'Tar:', 'none'] :plsr
For:CaCO3-> ['r2', 0.65, 'Spec:', 'log', 'bands:', 60, 'Tar:', 'minmax'] :svr


In [230]:
ipywidgets.interact( plot_model_acc, target = target_names, method = ml_methods, \
                    spec_preprocessing = prepare_spec,  \
                    target_preprocessing = prepare_target, n_bands = nbands_sampling )

interactive(children=(Dropdown(description='method', options=('mult', 'ridge', 'plsr', 'svr'), value='mult'), …

<function __main__.plot_model_acc(method, target, target_preprocessing, spec_preprocessing, n_bands)>

In [135]:
start = time.time()
grid_search = find_core_model_param_on ('plsr', 'Sand', 'none', 'none', 10)
end = time.time()
print('End time - Start time =', (end-start)) 

Fitting 20 folds for each of 10 candidates, totalling 200 fits
End time - Start time = 0.3782503604888916


In [139]:
start = time.time()
best_param = find_core_model_param_on ('gbrt', 'Sand', 'none', 'none', 10)
end = time.time()
print('End time - Start time =', (end-start)) 

Fitting 20 folds for each of 72 candidates, totalling 1440 fits
End time - Start time = 6.247332334518433


In [123]:
start = time.time()
best_param = find_core_model_param_on ('cubist', 'Sand', 'none', 'none', 10)
end = time.time()
print('End time - Start time =', (end-start)) 

Fitting 20 folds for each of 20 candidates, totalling 400 fits
End time - Start time = 7.174680948257446


In [74]:
start = time.time()
cubist_out = find_core_model_param_on ('cubist', 'Sand', 'log', 'minmax', 7)
end = time.time()
print('End time - Start time =', (end-start)) 

Fitting 500 folds for each of 20 candidates, totalling 10000 fits
End time - Start time = 113.41846990585327


In [127]:
start = time.time()
best_param = find_core_model_param_on ('ridge', 'Sand', 'none', 'none', 10)
end = time.time()
print('End time - Start time =', (end-start)) 

Fitting 20 folds for each of 14 candidates, totalling 280 fits
End time - Start time = 0.44140100479125977


In [130]:
start = time.time()
best_param = find_core_model_param_on ('svr', 'Sand', 'none', 'none', 10)
end = time.time()
print('End time - Start time =', (end-start)) 

Fitting 20 folds for each of 18 candidates, totalling 360 fits
End time - Start time = 0.5347926616668701


NameError: name 'halt_here' is not defined

# Step 3: Leave one out Predictions

##  find_y_pred

In [ ]:
def find_y_pred (spectra, target, method):
    m = method
    (r,c) = spectra.shape
    
    Y_test = target.copy()
    Y_pred = target.copy()
    
    n_comp = 'NA'
    
#     if method == 'plsr':
#         #--- avg_iqrp[n] stores iqrp for plsr when plsr model is created with 'n' n_comp... 
#         avg_iqrp =[0]
#         Yp =['NA']
#         for n in range (1,max_n_comp):
#             for i in range (0,r): 
#                 full_spec = spectra.copy()
#                 X_train = full_spec.drop(full_spec.index[i], axis=0)
#                 X_test = full_spec.iloc[[i],:].copy()
#                 full_tar = target.copy()
#                 y_train = full_tar.drop(full_tar.index[i], axis=0)
#                 y_test = full_tar.iloc[i].copy() 
            
#                 Model = PLSRegression(n_components=n, scale=True)
#                 Model.fit(X_train, y_train)
#                 y_pred = Model.predict(X_test, copy=True)
#                 Y_pred.iloc[i] = y_pred
                
                
#             avg_iqrp.append(np.round(find_iqrp (Y_pred, Y_test),2))
            
#             Yp.append(Y_pred)
        
#         max_iqrp = max(avg_iqrp)
#         n_comp = avg_iqrp.index(max_iqrp)
#         Y_pred = Yp[n_comp]
                
        
        
    if method == 'plsr':
        #--- avg_iqrp[n] stores iqrp for plsr when plsr model is created with 'n' n_comp... 
        avg_iqrp =[]
        Y_pred = []        
        N =[]
        
        for n in range (0,max_n_comp):
            avg_iqrp.append(0)
            Y_pred.append(target.copy())
            N.append(n)
        
        for i in range (0,r): 
            print('i: ', i)
            full_spec = spectra.copy()
            X_train = full_spec.drop(full_spec.index[i], axis=0)
            X_test = full_spec.iloc[[i],:].copy()
            full_tar = target.copy()
            y_train = full_tar.drop(full_tar.index[i], axis=0)
            y_test = full_tar.iloc[i].copy() 
            
            Model = list(map(lambda x: (PLSRegression(n_components=x+1, scale=True)), N))            
            Model = list(map(lambda x: Model[x].fit(X_train, y_train), N))            
            y_pred = list(map(lambda x:Model[x].predict(X_test, copy=True), N))                          
            for n in range (0,max_n_comp):
                Y_pred[n].iloc[i] = y_pred[n]
                                
                
        for n in range (0,max_n_comp):        
            avg_iqrp[n] = (np.round(find_iqrp (Y_pred[n], Y_test),2))
            
        Yp = Y_pred.copy()
        
        max_iqrp = max(avg_iqrp)
        n_comp = avg_iqrp.index(max_iqrp) + 1
        Y_pred = Yp[n_comp - 1]
                
            
    else:  
        for i in range (0,r): 
            #print('i: ', i)
            full_spec = spectra.copy()
            X_train = full_spec.drop(full_spec.index[i], axis=0)
            X_test = full_spec.iloc[[i],:].copy()
            full_tar = target.copy()
            y_train = full_tar.drop(full_tar.index[i], axis=0)
            y_test = full_tar.iloc[i].copy() 
            
            #----- MULTILINEAR:  fitting, and prediction---------- 
            if m == 'mult':
                Model = linear_model.LinearRegression()    
            #----- RANDOM_FOREST:   fitting, and prediction---------- 
            elif m == 'randomforest': 
                Model = RandomForestRegressor(random_state= 23)    
            #----- CUBIST REGRESSION:    fitting and prediction---------
            elif m == 'cubist':
                Model = Cubist(n_rules = 50, n_committees = 5, random_state = 42)    
            #------ SUPPORT VECTOR MACHINE FOR REGRESSION: fitting and prediction-----------      
            elif m == 'svr': 
                Model = SVR()
            #------ RIDGE REGRESSION: fitting and prediction-----------      
            elif m == 'ridge': 
                Model = KernelRidge(alpha=0.001)
            #------ LASSO REGRESSION: fitting and prediction-----------      
            elif m == 'lasso': 
                Model = Lasso()   
            #------ GRADIENT BOOSTING REGRESSION: fitting and prediction-----------      
            else: 
                Model = GradientBoostingRegressor()
                    
            Model.fit(X_train, y_train)
            y_pred = Model.predict(X_test)
            #print(y_pred.shape)
            Y_pred.iloc[i] = y_pred

    return (Y_pred, n_comp)

# Step 4: Building Model Tree (Mtree)

In [ ]:
import os
os.system('say "your Meta Tree started building"')
import time

In [ ]:
# 0. Available machine learning regression models --------------------------------- (7)
#ml_methods = ['mult', 'plsr', 'randomforest', 'svr', 'ridge', 'gbrt']
#ml_methods = ['mult', 'plsr', 'cubist', 'randomforest', 'ridge' 'gbrt', 'svr']

In [ ]:
def build_tree_on (method_name, target):
    tree ={}
    start = time.time()
    m = method_name
    t = target
    
    print('tree for: '+ m +' ------> running on: ' + t)
    
    for tp in prepare_target:
        tree[tp] ={}
        for p in prepare_spec:
            tree[tp][p] ={}
            for n in nbands_sampling:
                tree[tp][p][n] ={}
                Y = tree[tp][p][n]
                    
                        
                #------ setting spec to appropriate (sampled) spectra----
                if p == 'none':
                    if n == 0:
                        if m == 'randomforest' or m == 'cubist' or m == 'gbrt':
                            #---- reched here due to tree based methods e.g. randomforest--
                            spec = sampled_spec[100]
                        else:
                            spec = spec2[51]
                    else:
                        #---- reached here with some n!=0----
                         spec = sampled_spec[n]
                        
#                     elif p == 'fod':
#                         if n == 0:
#                             if m == 'randomforest' or m == 'cubist' or m == 'gbrt':
#                                 #---- reched here due to tree based methods e.g. randomforest--
#                                 spec = sampled_fod[100]
#                             else:
#                                 spec = fod_spec
#                         else:
#                             #---- reached here with some n!=0----
#                             spec = sampled_fod[n]
                elif p == 'fod2':
                    if n == 0:
                        if m == 'randomforest' or m == 'cubist' or m == 'gbrt':
                            #---- reched here due to tree based methods e.g. randomforest--
                            spec = fod_sampled[100]
                        else:
                            spec = fod_spec
                    else:
                        #---- reached here with some n!=0----
                        spec = fod_sampled[n]            
                elif p== 'continuum':  
                    if n == 0:
                        if m == 'randomforest' or m == 'cubist' or m == 'gbrt':
                            #---- reched here due to tree based methods e.g. randomforest--
                            spec = sampled_cr[100]
                        else:
                            spec = cr_spec
                    else:
                        #---- reached here with some n!=0----
                        spec = sampled_cr[n]
                        
                else: 
                    if n == 0:
                        if m == 'randomforest' or m == 'cubist' or m == 'gbrt':
                            #---- reched here due to tree based methods e.g. randomforest--
                            spec = sampled_log[100]
                        else:
                            spec = log_spec
                    else:
                        #---- reached here with some n!=0----
                        spec = sampled_log[n]
                    
                    
                    
                #---- target selection and normalization ---
                if tp == 'none':
                    y = T[target_names.index(t)]
                    rand_n = rand_t[target_names.index(t)]  #-- for future use in train-test split
                else:
                    y = NT[target_names.index(t)]
                    rand_n = rand_nt[target_names.index(t)] #-- for future use in train-test split
                    #print('one more target set')
                            
                #----- calling find_y_pred to compute leave one out predictions----
                y_pred, n_com = find_y_pred (spec, y, m)
                y_test = y
                    
                    
#                     print(y_test.shape)
#                     print(y_pred.shape)
                    
#                     if m == 'plsr':
#                         y_pred = y_pred[:,0]
                    
                #----- Model accuracy evaluation and plotting datas-----------
                            
                Y['test'] = y_test
                Y['testP'] = y_pred
                Y['n_comp'] = n_com
                    
                    
                    

                Y['iqrp_test'] = find_iqrp(y_pred, y_test)
                Y['r2_test'] = find_r2(y_pred, y_test)
                Y['rpd_test'] = find_rpd(y_pred, y_test)                    
                Y['rmse_test'] = find_rmse(y_pred, y_test)
                                                                
                                                                         
    end = time.time()                            
    os.system('say "your meta tree finished for one more method"')                            

    print('End time - Start time =', (end-start)) 
    
    return (tree.copy())

In [ ]:
def build_tree_for (method_name):
    tree ={}
    start = time.time()
    m = method_name
    #-- code to build tree----
    for t in target_names:
        print('tree for: '+ m +' ------> running on: ' + t)
        tree[t] ={}
        for tp in prepare_target:
            tree[t][tp] ={}
            for p in prepare_spec:
                tree[t][tp][p] ={}
                for n in nbands_sampling:
                    tree[t][tp][p][n] ={}
                    Y = tree[t][tp][p][n]
                    
                        
                    #------ setting spec to appropriate (sampled) spectra----
                    if p == 'none':
                        if n == 0:
                            if m == 'randomforest' or m == 'cubist' or m == 'gbrt':
                                #---- reched here due to tree based methods e.g. randomforest--
                                spec = sampled_spec[100]
                            else:
                                spec = spec2[51]
                        else:
                            #---- reached here with some n!=0----
                            spec = sampled_spec[n]
                        
#                     elif p == 'fod':
#                         if n == 0:
#                             if m == 'randomforest' or m == 'cubist' or m == 'gbrt':
#                                 #---- reched here due to tree based methods e.g. randomforest--
#                                 spec = sampled_fod[100]
#                             else:
#                                 spec = fod_spec
#                         else:
#                             #---- reached here with some n!=0----
#                             spec = sampled_fod[n]
                    elif p == 'fod2':
                        if n == 0:
                            if m == 'randomforest' or m == 'cubist' or m == 'gbrt':
                                #---- reched here due to tree based methods e.g. randomforest--
                                spec = fod_sampled[100]
                            else:
                                spec = fod_spec
                        else:
                            #---- reached here with some n!=0----
                            spec = fod_sampled[n]            
                    elif p== 'continuum':  
                        if n == 0:
                            if m == 'randomforest' or m == 'cubist' or m == 'gbrt':
                                #---- reched here due to tree based methods e.g. randomforest--
                                spec = sampled_cr[100]
                            else:
                                spec = cr_spec
                        else:
                            #---- reached here with some n!=0----
                            spec = sampled_cr[n]
                        
                    else: 
                        if n == 0:
                            if m == 'randomforest' or m == 'cubist' or m == 'gbrt':
                                #---- reched here due to tree based methods e.g. randomforest--
                                spec = sampled_log[100]
                            else:
                                spec = log_spec
                        else:
                            #---- reached here with some n!=0----
                            spec = sampled_log[n]
                    
                    
                    
                    #---- target selection and normalization ---
                    if tp == 'none':
                        y = T[target_names.index(t)]
                        rand_n = rand_t[target_names.index(t)]  #-- for future use in train-test split
                    else:
                        y = NT[target_names.index(t)]
                        rand_n = rand_nt[target_names.index(t)] #-- for future use in train-test split
                        #print('one more target set')
                            
                    #----- calling find_y_pred to compute leave one out predictions----
                    y_pred, n_com = find_y_pred (spec, y, m)
                    y_test = y
                    
                    
#                     print(y_test.shape)
#                     print(y_pred.shape)
                    
#                     if m == 'plsr':
#                         y_pred = y_pred[:,0]
                    
                    #----- Model accuracy evaluation and plotting datas-----------
                                
                    Y['test'] = y_test
                    Y['testP'] = y_pred
                    Y['n_comp'] = n_com
                    
                    
                    

                    Y['iqrp_test'] = find_iqrp(y_pred, y_test)
                    Y['r2_test'] = find_r2(y_pred, y_test)
                    Y['rpd_test'] = find_rpd(y_pred, y_test)
                    Y['rmse_test'] = find_rmse(y_pred, y_test)
                                                                
                                                                         
    end = time.time()                            
    os.system('say "your meta tree finished for one more method"')                            

    print('End time - Start time =', (end-start)) 
    
    return (tree.copy())


## Loading the saved Mtree

In [ ]:
with open ('L1Mtree.pickle', 'rb') as file:
    Mtree = pickle.load(file)
    

## Best of all worlds

In [ ]:
#ml_methods = ['mult', 'plsr', 'randomforest','svr', 'ridge', 'gbrt']

In [ ]:
def best_model_parameters (Mtree, target, method, scorer):
    t=target
    m= method
    
    best_score = -1
    best_n_comp = 'NA'
    
    
    for tp in prepare_target:
        for p in prepare_spec:
            for n in nbands_sampling:
                Y = Mtree[m][t][tp][p][n]
                #r2_train = Y['r2_train']
                r2_test = Y['r2_test']
                    
                if scorer == 'iqrp':
                    cur_score = Y['iqrp_test']
                elif scorer == 'rpd':
                    cur_score = Y['rpd_test']
                else:
                    cur_score = Y['r2_test']  
                    
                if cur_score > best_score:
                    best_score = cur_score
                    best_tp = tp
                    best_n = n
                    best_p = p
                    if m == 'plsr':
                        best_n_comp = Y['n_comp']
                            
    param_list = [scorer, np.round(best_score,2), 'Spec:', best_p, 'bands:', best_n, 'Tar:', best_tp]                                 
    return (param_list)                                
    

In [ ]:
def best_score_for (Mtree, target, scorer):
    
    for method in ml_methods:
        param_list= best_model_parameters (Mtree, target, method, scorer)
        print('For:'+target+'->', param_list, ':'+method)
     
    return

## Plotting Model Accuracy (ipywidgets)

## Creating different branches of Mtree (for separate methods)

In [ ]:
# Mtree Building (run below code only once)

#Mtree ={}

In [ ]:
# max_n_comp = 10
# Mtree['plsr'] = build_tree_for ('plsr')

# Mtree['mult'] = build_tree_for ('mult')

# Mtree['svr'] = build_tree_for ('svr')

In [ ]:
#Mtree['ridge']['Sand'] = build_tree_on ('ridge', 'Sand')

In [ ]:
#Mtree['ridge'] = build_tree_for ('ridge')

In [ ]:
#Mtree['lasso'] = build_tree_for ('lasso')

In [ ]:
#Mtree['cubist'] = build_tree_for ('cubist')

In [ ]:
# with open ('L1Mtree.pickle', 'wb') as file:
#     pickle.dump(Mtree, file)

In [ ]:
#Mtree['gbrt'] = build_tree_for ('gbrt')

In [ ]:
# with open ('L1Mtree.pickle', 'wb') as file:
#     pickle.dump(Mtree, file)

In [ ]:
#Mtree['randomforest'] = build_tree_for ('randomforest')

In [ ]:
# with open ('L1Mtree.pickle', 'wb') as file:
#     pickle.dump(Mtree, file)

In [ ]:
best_score_for(Mtree, 'Sand', 'r2')

In [ ]:
best_score_for(Mtree, 'Sand', 'iqrp')

In [ ]:
best_score_for(Mtree, 'Silt', 'r2')

In [ ]:
best_score_for(Mtree, 'Silt', 'iqrp')

In [ ]:
best_score_for(Mtree, 'Clay', 'r2')

In [ ]:
best_score_for(Mtree, 'Clay', 'iqrp')

In [ ]:
best_score_for(Mtree, 'TOC', 'r2')

In [ ]:
best_score_for(Mtree, 'TOC', 'iqrp')

In [ ]:
best_score_for(Mtree, 'CaCO3', 'r2')

In [ ]:
best_score_for(Mtree, 'CaCO3', 'iqrp')

In [ ]:
ipywidgets.interact( plot_model_acc, target = target_names, method = ml_methods, \
                    spec_preprocessing = prepare_spec,  \
                    target_preprocessing = prepare_target, n_bands = nbands_sampling )

In [ ]:
# with open ('L1Mtree.pickle', 'wb') as file:
#     pickle.dump(Mtree, file)

In [ ]:
from sklearn.kernel_ridge import KernelRidge
krr_reg = KernelRidge()

param_grid={'alpha': [ 0.00001, 0.00005, 0.00002, 0.00025,0.001, 0.005, 0.0001, 0.0005, 0.01, 0.05, 0.1, 0.5, 1]}

scorer = make_scorer(mean_squared_error, greater_is_better=False)
cv = RepeatedKFold(n_splits=10, n_repeats=10, random_state=3)
krr_random = GridSearchCV(krr_reg, param_grid=param_grid, cv=cv,\
                                scoring=scorer, refit=True, verbose=1, error_score='raise', n_jobs=-1)

krr_random.fit(sampled_log[7],NT[0])

In [ ]:
print(krr_random.best_estimator_)
# # y_pred = krr_random.predict(X_test)
# # y_pred = np.round(y_pred, 2)

# # score_cv = r2_score(y_test, y_pred)


